In [1]:
import os

os.chdir('..')
import glob
import re

import numpy as np
import pandas as pd
import tqdm

pd.set_option("display.max_columns", None)


In [2]:
def join_games(file_list):
   
   df = pd.DataFrame()

   for match in tqdm.tqdm(file_list, ncols=100):

      clean_match = match.split("\\")
      # print(' '.join(clean_match))

      year = int(clean_match[0].split("/")[1])
      jornada = int(clean_match[1].split()[-1])
      local = clean_match[-1].split(" vs ")[0]
      visit = clean_match[-1].split(" vs ")[1].replace(".xlsx", "")
      game = clean_match[-1].replace(".xlsx", "")
      # print(year, jornada, local, visit)

      df_local = pd.read_excel(match, sheet_name=local)
      df_local["Equipo"] = local
      df_local["Estado"] = 'Local'
      df_local["Siglas"] = re.sub("[^A-Z]", "", local)
      df_local["Jornada"] = jornada
      df_local["Anio"] = year
      df_local["Game"] = game

      df_visit = pd.read_excel(match, sheet_name=visit)
      df_visit["Equipo"] = visit
      df_visit["Estado"] = 'Visitante'
      df_visit["Siglas"] = re.sub("[^A-Z]", "", visit)
      df_visit["Jornada"] = jornada
      df_visit["Anio"] = year
      df_visit["Game"] = game

      df = pd.concat([df, df_local, df_visit], ignore_index=True)

   df.columns = ['Jugador', 'P', 'M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
                    'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS',
                    'Equipo', 'Estado', 'Siglas', 'Jornada', 'Anio', 'Game']

   df.PPC = df.PPC.replace({'%':''}, regex=True)
   num_columns = ['M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS']

   df[num_columns] = df[num_columns].apply(pd.to_numeric, errors='coerce', axis=1)
    
   return df


In [3]:

excels_2021 = glob.glob("data/2021/*/*.xlsx")


In [4]:

df_2021 = join_games(excels_2021)

100%|█████████████████████████████████████████████████████████████| 153/153 [00:06<00:00, 24.11it/s]


# EDA 2021

In [5]:
df_2021.head()

,Jugador,P,M,V,AG,AST,A,R,PENALTI,PP,PF,D,DP,D1,DS,Rob,P1,PC,PPC,D2,FC,FS,Equipo,Estado,Siglas,Jornada,Anio,Game
0,Christian Limousin,P,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,8.0,38.0,2.0,0.0,0.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
1,Diego Otoya,D,90.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,32.0,28.0,88.0,0.0,2.0,1.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
2,José Ramírez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,36.0,32.0,89.0,0.0,2.0,1.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
3,Joseph Vega,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,18.0,15.0,83.0,0.0,1.0,0.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
4,Gianmarco Gambetta,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,35.0,23.0,66.0,0.0,3.0,0.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal


In [6]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5493 entries, 0 to 5492
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Jugador  5493 non-null   object 
 1   P        5493 non-null   object 
 2   M        5493 non-null   float64
 3   V        5493 non-null   float64
 4   AG       5493 non-null   float64
 5   AST      5493 non-null   float64
 6   A        5493 non-null   float64
 7   R        5493 non-null   float64
 8   PENALTI  5493 non-null   float64
 9   PP       5493 non-null   float64
 10  PF       5493 non-null   float64
 11  D        5134 non-null   float64
 12  DP       5134 non-null   float64
 13  D1       5134 non-null   float64
 14  DS       5134 non-null   float64
 15  Rob      5134 non-null   float64
 16  P1       5134 non-null   float64
 17  PC       5134 non-null   float64
 18  PPC      5134 non-null   float64
 19  D2       5134 non-null   float64
 20  FC       5134 non-null   float64
 21  FS       5134 

## Eliminando partidos con registros nulos

In [7]:
nan_rows = df_2021[df_2021.isnull().any(axis=1)].copy().reset_index(drop=True)
# nan_jornada_games_2021 = nan_rows.groupby(['Jornada','Game']).size().reset_index(name='Cantidad')
# print(nan_jornada_games_2021)
drop_games_2021 = nan_rows.Game.unique()
drop_games_2021
# nan_rows.Game.unique()

array(['Binacional vs Alianza Huánuco',
       'Sport Huancayo vs Alianza Huánuco',
       'Univ. César Vallejo vs Alianza Huánuco',
       'Alianza Atlético vs Deportivo Municipal',
       'Carlos A. Mannucci vs Academia Cantolao',
       'Cusco FC vs Sport Huancayo', 'Cusco FC vs Alianza Huánuco',
       'Alianza Atlético vs Alianza Huánuco',
       'Alianza Huánuco vs Univ. San Martín',
       'Univ. Técnica vs Alianza Huánuco'], dtype=object)

In [8]:
df_2021.columns

Index(['Jugador', 'P', 'M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS',
       'Equipo', 'Estado', 'Siglas', 'Jornada', 'Anio', 'Game'],
      dtype='object')

In [9]:
df_2021 = df_2021.loc[~df_2021.Game.isin(drop_games_2021)].reset_index(drop=True)

In [10]:
df_2021.head()

,Jugador,P,M,V,AG,AST,A,R,PENALTI,PP,PF,D,DP,D1,DS,Rob,P1,PC,PPC,D2,FC,FS,Equipo,Estado,Siglas,Jornada,Anio,Game
0,Christian Limousin,P,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,8.0,38.0,2.0,0.0,0.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
1,Diego Otoya,D,90.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,32.0,28.0,88.0,0.0,2.0,1.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
2,José Ramírez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,36.0,32.0,89.0,0.0,2.0,1.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
3,Joseph Vega,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,18.0,15.0,83.0,0.0,1.0,0.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal
4,Gianmarco Gambetta,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,35.0,23.0,66.0,0.0,3.0,0.0,Academia Cantolao,Local,AC,1,2021,Academia Cantolao vs Sporting Cristal


In [11]:
df_2021.columns

Index(['Jugador', 'P', 'M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS',
       'Equipo', 'Estado', 'Siglas', 'Jornada', 'Anio', 'Game'],
      dtype='object')

In [12]:
sum_columns = ['V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'D2', 'FC', 'FS']

df_2021_groupby = df_2021.groupby(['Game', 'Jornada','Estado'])[sum_columns].sum().sort_values('Game').reset_index().copy()
df_2021_groupby


,Game,Jornada,Estado,V,AG,AST,A,R,PENALTI,PP,PF,D,DP,D1,DS,Rob,P1,PC,D2,FC,FS
0,Academia Cantolao vs Ayacucho FC,3,Local,2.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,10.0,7.0,10.0,6.0,37.0,275.0,181.0,5.0,20.0,16.0
1,Academia Cantolao vs Ayacucho FC,3,Visitante,2.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,23.0,7.0,15.0,11.0,6.0,415.0,323.0,5.0,18.0,19.0
2,Academia Cantolao vs Binacional,12,Local,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,10.0,3.0,17.0,13.0,12.0,384.0,277.0,7.0,17.0,20.0
3,Academia Cantolao vs Binacional,12,Visitante,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,24.0,9.0,12.0,11.0,11.0,377.0,284.0,1.0,21.0,17.0
4,Academia Cantolao vs Deportivo Municipal,8,Local,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,4.0,15.0,11.0,21.0,319.0,221.0,1.0,22.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,Universitario vs Cusco FC,16,Visitante,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,11.0,5.0,20.0,13.0,15.0,365.0,269.0,4.0,19.0,12.0
282,Universitario vs Deportivo Municipal,11,Local,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,14.0,6.0,16.0,11.0,5.0,375.0,259.0,4.0,12.0,13.0
283,Universitario vs Deportivo Municipal,11,Visitante,2.0,0.0,1.0,5.0,0.0,1.0,0.0,0.0,12.0,6.0,16.0,8.0,24.0,366.0,254.0,6.0,16.0,11.0
284,Universitario vs Univ. César Vallejo,14,Local,3.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,8.0,5.0,28.0,21.0,16.0,251.0,160.0,3.0,19.0,19.0


In [13]:
games_sum = [df_2021_groupby.loc[i:i+1, :] for i in range(0, len(df_2021_groupby), 2)]

In [14]:
games_sum = [df_2021_groupby.loc[i:i+1, :] for i in range(0, len(df_2021_groupby), 2)]

df_2021 = pd.DataFrame()
for dataframe in games_sum:

    dataframe = dataframe.sort_values(['Jornada', 'Game', 'Estado'], ignore_index=True)
    
    temp_1 = pd.DataFrame(dataframe.iloc[0]).T.reset_index(drop=True)
    temp_1.drop(columns=['Estado', 'Jornada'], inplace=True)
    temp_1.columns = [col+'_L' for col in temp_1.columns]
    temp_1.rename(columns={'Game_L':'Game'}, inplace=True)

    temp_2 = pd.DataFrame(dataframe.iloc[1]).T.reset_index(drop=True)
    temp_2.drop(columns=['Game','Jornada','Estado'], inplace=True)
    temp_2.columns = [col+'_V' for col in temp_2.columns]

    merge_temp = pd.concat([temp_1, temp_2], axis=1)
    
    df_2021 = pd.concat([df_2021, merge_temp], ignore_index=True)

In [15]:
df_2021.head()

,Game,V_L,AG_L,AST_L,A_L,R_L,PENALTI_L,PP_L,PF_L,D_L,DP_L,D1_L,DS_L,Rob_L,P1_L,PC_L,D2_L,FC_L,FS_L,V_V,AG_V,AST_V,A_V,R_V,PENALTI_V,PP_V,PF_V,D_V,DP_V,D1_V,DS_V,Rob_V,P1_V,PC_V,D2_V,FC_V,FS_V
0,Academia Cantolao vs Ayacucho FC,2.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,10.0,7.0,10.0,6.0,37.0,275.0,181.0,5.0,20.0,16.0,2.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,23.0,7.0,15.0,11.0,6.0,415.0,323.0,5.0,18.0,19.0
1,Academia Cantolao vs Binacional,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,10.0,3.0,17.0,13.0,12.0,384.0,277.0,7.0,17.0,20.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,24.0,9.0,12.0,11.0,11.0,377.0,284.0,1.0,21.0,17.0
2,Academia Cantolao vs Deportivo Municipal,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,4.0,15.0,11.0,21.0,319.0,221.0,1.0,22.0,18.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,6.0,2.0,22.0,13.0,18.0,312.0,212.0,2.0,19.0,22.0
3,Academia Cantolao vs FBC Melgar,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,18.0,12.0,18.0,301.0,203.0,3.0,13.0,17.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,19.0,5.0,18.0,14.0,14.0,424.0,341.0,0.0,18.0,12.0
4,Academia Cantolao vs Sport Boys Callao,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,15.0,1.0,18.0,17.0,20.0,363.0,283.0,1.0,11.0,21.0,2.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,11.0,3.0,20.0,16.0,24.0,359.0,282.0,1.0,21.0,11.0


In [16]:
games_2021 = pd.read_pickle('data/2021_stats_final.pkl')
result = pd.merge(games_2021, df_2021, on="Game")

In [17]:
result.to_parquet('stats_2021.parquet')

In [18]:
test = pd.read_parquet('stats_2021.parquet')
test

,Jornada,Fecha,Local,Visitante,Goles Local,Goles Visitante,Link,Total despejes local,Total despejes visitante,Disparos fuera local,Disparos fuera visitante,Tiros local,Tiros visitante,Faltas local,Faltas visitante,Total tackles: local,Total tackles: visitante,Saque de meta local,Saque de meta visitante,Duelos ganados local,Duelos ganados visitante,Posesión local,Posesión visitante,Saques de banda local,Saques de banda visitante,Remates a puerta local,Remates a puerta visitante,Fueras de juego local,Fueras de juego visitante,Paradas local,Paradas visitante,Total de pases local,Total de pases visitante,Pases acertados local,Pases acertados visitante,Disparos bloqueados local,Disparos bloqueados visitante,Saques de esquina local,Saques de esquina visitante,Faltas sufridas local,Faltas sufridas visitante,Game,V_L,AG_L,AST_L,A_L,R_L,PENALTI_L,PP_L,PF_L,D_L,DP_L,D1_L,DS_L,Rob_L,P1_L,PC_L,D2_L,FC_L,FS_L,V_V,AG_V,AST_V,A_V,R_V,PENALTI_V,PP_V,PF_V,D_V,DP_V,D1_V,DS_V,Rob_V,P1_V,PC_V,D2_V,FC_V,FS_V
0,1,2020-07-17,Deportivo Municipal,Univ. Técnica,1,1,https://www.ceroacero.es/match.php?id=8066759,21,17,5,0,8,5,16,22,10,15,6,7,7,9,48 %,52 %,22,32,2,4,1,1,3,1,329,354,226,249,1,1,8,2,21,16,Deportivo Municipal vs Univ. Técnica,1.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,8.0,2.0,10.0,7.0,21.0,329.0,226.0,3.0,16.0,21.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,5.0,4.0,15.0,9.0,17.0,354.0,249.0,1.0,22.0,16.0
1,1,2020-07-18,Univ. San Martín,Carlos A. Mannucci,0,2,https://www.ceroacero.es/match.php?id=8066760,16,18,2,4,9,17,13,15,14,10,8,5,14,9,55 %,45 %,15,14,5,7,4,4,5,5,436,343,344,265,2,6,2,13,14,13,Univ. San Martín vs Carlos A. Mannucci,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,9.0,5.0,14.0,14.0,16.0,436.0,344.0,5.0,13.0,14.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,17.0,7.0,10.0,9.0,18.0,343.0,265.0,5.0,15.0,13.0
2,1,2020-07-18,Academia Cantolao,Sporting Cristal,2,4,https://www.ceroacero.es/match.php?id=8066761,9,12,3,3,10,10,23,14,11,10,4,7,7,7,41 %,59 %,16,11,7,6,1,4,2,5,289,431,232,363,0,1,4,3,13,23,Academia Cantolao vs Sporting Cristal,2.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,10.0,7.0,11.0,7.0,9.0,289.0,232.0,2.0,23.0,13.0,4.0,0.0,3.0,3.0,0.0,1.0,0.0,0.0,10.0,6.0,10.0,7.0,12.0,431.0,363.0,5.0,14.0,23.0
3,1,2020-07-18,Sport Huancayo,FBC Melgar,0,3,https://www.ceroacero.es/match.php?id=8066762,12,27,5,6,12,14,15,29,12,22,3,7,7,16,60 %,41 %,30,13,5,4,2,2,1,5,448,312,337,218,2,4,7,3,28,12,Sport Huancayo vs FBC Melgar,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,12.0,5.0,12.0,7.0,12.0,448.0,337.0,1.0,15.0,28.0,3.0,0.0,1.0,3.0,0.0,2.0,0.0,0.0,14.0,4.0,22.0,16.0,27.0,312.0,218.0,5.0,29.0,12.0
4,1,2020-07-19,Sport Boys Callao,Cienciano,1,4,https://www.ceroacero.es/match.php?id=8066763,17,15,3,5,9,20,11,13,11,24,7,10,3,19,58 %,42 %,26,19,3,8,1,1,4,2,511,369,423,291,3,7,4,6,10,11,Sport Boys Callao vs Cienciano,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,9.0,3.0,11.0,3.0,17.0,511.0,423.0,4.0,11.0,10.0,4.0,0.0,3.0,4.0,0.0,1.0,0.0,0.0,20.0,8.0,24.0,19.0,15.0,369.0,291.0,2.0,13.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,17,2020-10-30,Sporting Cristal,Sport Boys Callao,1,1,https://www.ceroacero.es/match.php?id=8066909,7,9,2,3,6,10,13,9,6,22,6,8,4,13,65 %,35 %,16,18,2,4,None,None,3,1,595,310,514,233,2,3,3,2,9,12,Sporting Cristal vs Sport Boys Callao,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0,2.0,6.0,4.0,7.0,595.0,514.0,3.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0,4.0,22.0,13.0,9.0,310.0,233.0,1.0,9.0,12.0
139,17,2020-10-30,Binacional,Univ. César Vallejo,0,0,https://www.ceroacero.es/match.php?id=8066911,12,37,8,1,16,4,12,15,18,11,4,8,17,7,33 %,67 %,23,26,4,2,3,3,2,4,273,583,167,491,4,1,8,3,15,12,Binacional vs Univ. César Vallejo,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16.0,4.0,18.0,17.0,12.0,273.0,167.0,2.0,12.0,15.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,2.0,11.

In [19]:
test.loc[test.Game=='Academia Cantolao vs Ayacucho FC']

,Jornada,Fecha,Local,Visitante,Goles Local,Goles Visitante,Link,Total despejes local,Total despejes visitante,Disparos fuera local,Disparos fuera visitante,Tiros local,Tiros visitante,Faltas local,Faltas visitante,Total tackles: local,Total tackles: visitante,Saque de meta local,Saque de meta visitante,Duelos ganados local,Duelos ganados visitante,Posesión local,Posesión visitante,Saques de banda local,Saques de banda visitante,Remates a puerta local,Remates a puerta visitante,Fueras de juego local,Fueras de juego visitante,Paradas local,Paradas visitante,Total de pases local,Total de pases visitante,Pases acertados local,Pases acertados visitante,Disparos bloqueados local,Disparos bloqueados visitante,Saques de esquina local,Saques de esquina visitante,Faltas sufridas local,Faltas sufridas visitante,Game,V_L,AG_L,AST_L,A_L,R_L,PENALTI_L,PP_L,PF_L,D_L,DP_L,D1_L,DS_L,Rob_L,P1_L,PC_L,D2_L,FC_L,FS_L,V_V,AG_V,AST_V,A_V,R_V,PENALTI_V,PP_V,PF_V,D_V,DP_V,D1_V,DS_V,Rob_V,P1_V,PC_V,D2_V,FC_V,FS_V
18,3,2020-08-01,Academia Cantolao,Ayacucho FC,2,2,https://www.ceroacero.es/match.php?id=8066784,37,6,2,12,10,23,20,18,10,15,13,5,6,11,38 %,62 %,15,20,7,7,4,5,5,5,275,415,181,323,1,4,3,6,16,19,Academia Cantolao vs Ayacucho FC,2.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,10.0,7.0,10.0,6.0,37.0,275.0,181.0,5.0,20.0,16.0,2.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,23.0,7.0,15.0,11.0,6.0,415.0,323.0,5.0,18.0,19.0
